In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from math import sqrt, atan, sin , cos

In [ ]:
PATH_SVG = "C:\\Users\\XAVIER\\3D Objects\\abadia\\abadia.svg"
LAYER_SVG = "layer1"

In [ ]:
# Posiciones referenciales de iconos
# obtener desde consola usando
# import pyautogui
# pyautogui.displayMousePosition()

POS_ICONO_FIRESTORM = (809, 1055)
POS_NUEVA_CAJA = (584, 234)

In [ ]:
import pyautogui as gui

def gui_click(imagen):
    if type(imagen) in [str, tuple]:
        imagen = [imagen]
    for img in imagen:
        if type(img) is tuple:
            gui.click( img[0], img[1])
            return img
        posicion = gui.locateOnScreen(img)
        if not posicion:
            continue
        x, y, ancho, alto = posicion 
        gui.click( x + ancho / 2, y + alto / 2)
        return ( x + ancho / 2, y + alto / 2)
    return False

def crear_caja( x, y, z, ancho, alto, espesor, angulo_z):
    # Paso 1    
    if not gui_click(['ui/icono_firestorm.png', POS_ICONO_FIRESTORM]):
        return False
    sleep(0.5)
    
    # Paso 2
    gui.hotkey('ctrl', 'b')
    sleep(0.5)
    
    # Paso 3
    if not gui_click(['ui/nueva_caja.png','ui/nueva_caja1.png','ui/nueva_caja2.png', POS_NUEVA_CAJA]):
        return False
    sleep(0.5)
    
    # Paso 3 (continuacion)
    mouse_x, mouse_y = gui.position()
    gui.moveTo(mouse_x - 50, mouse_y +200 )
    sleep(2.0)
    gui.click()

    sleep(2.0)
    
    # Paso 4
    #if not gui_click(['ui/objeto_tab.png','ui/objeto_tab1.png','ui/objeto_tab2.png','ui/objeto_tab3.png']):
    mouse_x, mouse_y = gui.position()
    gui.moveTo(mouse_x + 125, mouse_y - 15 )
    sleep(2.0)
    gui.click()
        #return False
    sleep(0.5)
    
    # Pasos 5, 6, 7 y 8
    gui.press('tab')
    gui.press('tab')
    gui.press('tab')
    gui.press('tab')
    gui.write('{0:.5f}'.format(x))
    gui.press('tab')
    gui.write('{0:.5f}'.format(y))
    gui.press('tab')
    gui.write('{0:.5f}'.format(z))
    gui.press('tab')
    gui.write('{0:.5f}'.format(ancho))
    gui.press('tab')
    gui.write('{0:.5f}'.format(alto))
    gui.press('tab')
    gui.write('{0:.5f}'.format(espesor))
    gui.press('tab')
    gui.press('tab')
    gui.press('tab')
    gui.write('{0:.5f}'.format(angulo_z))
    sleep(3.0)
    
    # Paso 9
    gui.hotkey('ctrl', 'b')
    sleep(0.5)
    
    # Paso 10
    gui.hotkey('alt', 'tab')
    sleep(0.5)
    print("Caja creada en posicion <{0:.5f}, {1:.5f}, {2:.5f}> ; tamaño: {3:.5f} x {4:.5f} x {5:.5f} ; angulo: {6:.5f}".format( x, y, z, ancho, alto, espesor, angulo_z) )
    sleep(1.0)
    return True

rpa.init(visual_automation = True, chrome_browser = False)
crear_caja( x = 133, y = 220, z = 23 , ancho = 3, alto = 2 , espesor = 1, angulo_z = 0)
crear_caja( x = 130, y = 220, z = 22 , ancho = 1, alto = 2.5 , espesor = 0.1, angulo_z = 30)
crear_caja( x = 135, y = 220, z = 22 , ancho = 2, alto = 1 , espesor = 0.75, angulo_z = 30)

print('Secuencia finalizada')
rpa.close()

In [ ]:
def importar_svg(nombre_capa = 'layer7', path_archivo = 'ChullaQuito.svg'):
    archivo = open( path_archivo, 'rt')    
    sopa = BeautifulSoup(archivo,'xml')
    archivo.close()
    capa = sopa.svg.find('g', id = nombre_capa)
    rectangulos = capa.find_all('rect')
    rectangulos
    valores = [ [ x['x'], x['y'], x['width'], x['height'], x['transform'] if 'transform' in x.attrs else 'rotate(0)'  ] for x in rectangulos ]
    valores = [ [ float(x[0]), float(x[1]), float(x[2]), float(x[3]), float(x[4][7:-1]) ] for x in valores ]
    return np.array(valores)

In [ ]:
importar_svg( nombre_capa = LAYER_SVG, path_archivo = PATH_SVG )

In [ ]:
def adaptar_coordenadas( vector ):
    # return [ vector[0] + vector[2] / 2, 256.0 - vector[1] - vector[3] / 2, vector[2], vector[3], - vector[4]  ]
    radio = sqrt( (vector[0] + vector[2] / 2) ** 2 + (vector[1] + vector[3] / 2 ) ** 2 )
    angulo_original_grados = atan( (vector[1] + vector[3] / 2 ) / (vector[0] + vector[2] / 2) ) * 180 / np.pi
    # angulo_original_grados = atan( (vector[1] + vector[3] / 2 ) / radio ) * 180 / np.pi
    nuevo_angulo_rads = (angulo_original_grados + vector[4]) * np.pi / 180
    vector_0 = abs(radio * cos(nuevo_angulo_rads))
    vector_1 = abs(radio * sin(nuevo_angulo_rads))
    return [ vector_0 , 256.0 - vector_1 , vector[2], vector[3], - vector[4]  ]
    #return [ vector_0 , vector_1 , vector[2], vector[3], - vector[4]  ]

In [ ]:
def construir_bloques(path_archivo = 'ChullaQuito.svg', nombre_capa = 'layer7', pendientes_anteriores = None, 
                      altura = 25 , espesor = 0.1 ):
    data = []
    if not pendientes_anteriores is None:
        data = pendientes_anteriores
    else:
        data = importar_svg( nombre_capa = nombre_capa, path_archivo = path_archivo )
        data = [ adaptar_coordenadas(x) for x in data]
    pendientes = []
    #rpa.init(visual_automation = True, chrome_browser = False)
    print('Iniciando secuencia');
    secuencia_activa = True
    for bloque in data:
        if secuencia_activa == False:
            pendientes.append(bloque)
            continue
        if not crear_caja( bloque[0], bloque[1], altura, bloque[2], bloque[3], espesor, bloque[4] ):
            print('Secuencia interrumpida')
            pendientes.append(bloque)
            secuencia_activa = False
    print('Secuencia finalizada')
    #rpa.close()
    if len(pendientes) == 0:
        pendientes = None
    return pendientes_anteriores

In [ ]:
pendientes = None

In [ ]:
# Construir el piso
pendientes = construir_bloques(path_archivo = PATH_SVG, nombre_capa = LAYER_SVG, 
                               pendientes_anteriores = pendientes, altura = 66.0, espesor = 4.0)
if not pendientes is None:
    print("quedan {0} pendientes".format(len(pendientes)))

In [ ]:
data = importar_svg( nombre_capa = LAYER_SVG, path_archivo = PATH_SVG )
data = [ [y for y in x ] + adaptar_coordenadas(x) for x in data ]
data = [ {'x':x[5], 'y':x[6], 'width':x[7], 'height':x[8], 'rotation':x[9], 'x1':x[0], 'y2':x[1], 'width1':x[2], 'height1':x[3], 'rotation1':x[4], } for x in data ]
##for x in data:
data = pd.DataFrame(data)
data
#vector_x = [[1,2], [4,5]]
#vector_y = [[4,5]]
#vector_z = [ x + [sum(x)] for x in vector_x ]
#vector_z

In [ ]:
adaptar_coordenadas([-62.876225, 99.632347, 0.2, 6.2879076,-40.217861])

In [ ]:
adaptar_coordenadas([-62.876225, 99.632347, 0.2, 6.2879076,0])

In [ ]:
print('{0} a {1}\n{2} a {3}'.format(data['x'].min(), data['x'].max(), data['y'].min(), data['y'].max())) 